# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [2]:
import pandas as pd
import numpy as np
import scipy.stats as st
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

Загрузим данные из трёх скважин:

In [3]:
try:
    well_1 = pd.read_csv("data_1.csv")
except:
    print("Could not open the file data_1")
try:
    well_2 = pd.read_csv("data_2.csv")
except:
    print("Could not open the file data_2")
try:
    well_3 = pd.read_csv("data_3.csv")
except:
    print("Could not open the file data_3")

напишем функцию для просмотра входящих данных

In [4]:
def checking_the_input_data(data):
    print(data.head())
    print()
    print("shape", data.shape)
    print()
    print("isna:", data.isna().sum())
    print()
    print("duplicated:", data.duplicated().sum())


In [5]:
checking_the_input_data(well_1)

      id        f0        f1        f2     product
0  txEyH  0.705745 -0.497823  1.221170  105.280062
1  2acmU  1.334711 -0.340164  4.365080   73.037750
2  409Wp  1.022732  0.151990  1.419926   85.265647
3  iJLyR -0.032172  0.139033  2.978566  168.620776
4  Xdl7t  1.988431  0.155413  4.751769  154.036647

shape (100000, 5)

isna: id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

duplicated: 0


In [6]:
checking_the_input_data(well_2)

      id         f0         f1        f2     product
0  kBEdx -15.001348  -8.276000 -0.005876    3.179103
1  62mP7  14.272088  -3.475083  0.999183   26.953261
2  vyE1P   6.263187  -5.948386  5.001160  134.766305
3  KcrkZ -13.081196 -11.506057  4.999415  137.945408
4  AHL4O  12.702195  -8.147433  5.004363  134.766305

shape (100000, 5)

isna: id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

duplicated: 0


In [7]:
checking_the_input_data(well_3)

      id        f0        f1        f2     product
0  fwXo0 -1.146987  0.963328 -0.828965   27.758673
1  WJtFt  0.262778  0.269839 -2.530187   56.069697
2  ovLUW  0.194587  0.289035 -5.586433   62.871910
3  q6cA6  2.236060 -0.553760  0.930038  114.572842
4  WPMUX -0.515993  1.716266  5.899011  149.600746

shape (100000, 5)

isna: id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

duplicated: 0


Данные геологоразведки всех трёх скважин имеют одинаковые размеры, не имеют пропусков и дубликатов

## Обучение и проверка модели

### Разобьём данные каждой скважины на обучающую и валидационную выборки в соотношении 75:25. Обучим модель линейной регрессии и сделаем предсказания на валидационной выборке. Найдём RMSE и R2 модели каждой скважины.  Для всего этого напишем функцию rmse_r2, которая будет возвращать предсказания, правильные ответы на валидационной выборке, а также RMSE и R2.

In [8]:
def rmse_r2(data):
    target = data['product']
    features = data.drop(['product',"id"], axis = 1)
    features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size = 0.25, random_state = 1234)
    model = LinearRegression()
    model.fit(features_train, target_train)
    predict = model.predict(features_valid)
    rmse = mean_squared_error(target_valid, predict)**.5
    r2 = r2_score(target_valid, predict)
    return target_valid, predict, rmse, r2


Посмотрим RMSE и R2 каждой скважины

In [9]:
w1_target_valid, w1_predict, w1_rmse, w1_r2 = rmse_r2(well_1)
print("RMSE первой скважины:", w1_rmse)
print("R-Squared первой скважины", w1_r2)

print("****************************")
w2_target_valid, w2_predict, w2_rmse, w2_r2 = rmse_r2(well_2)
print("RMSE второй скважины:", w2_rmse)
print("R-Squared второй скважины", w2_r2)

print('****************************')
w3_target_valid, w3_predict, w3_rmse, w3_r2 = rmse_r2(well_3)
print("RMSE третьей скважины:", w3_rmse)
print("R-Squared третьей скважины", w3_r2)


RMSE первой скважины: 37.562394183637785
R-Squared первой скважины 0.2740586761361865
****************************
RMSE второй скважины: 0.8939344738117703
R-Squared второй скважины 0.999619058765352
****************************
RMSE третьей скважины: 40.10318099258418
R-Squared третьей скважины 0.1983301043994069


### Напечатаем на экране средний запас предсказанного сырья и RMSE модели.

In [10]:
aver_rmse = pd.DataFrame(index=["Cредний запас предсказанного сырья", "RMSE модели" ], columns = ["скважина_1","скважина_2", 'скважина_3'])
aver_rmse["скважина_1"] = w1_predict.mean(), w1_rmse
aver_rmse["скважина_2"] = w2_predict.mean(), w2_rmse
aver_rmse["скважина_3"] = w3_predict.mean(), w3_rmse
aver_rmse

,скважина_1,скважина_2,скважина_3
Cредний запас предсказанного сырья,92.430460,68.802482,94.915738
RMSE модели,37.562394,0.893934,40.103181


Мы разбили данные по каждой скважине на обучающую и валидационную выборки в пропорции 75:25, обучили на них модель линейной регрессии и сделали предсказания. Для модели каждого региона были найдена метрика RMSE. Для второго региона величина среднеквадратичной ошибки оказалась сильно меньше двух других и составила 0,894 против 37,562 для первого и 40,103 для третьего.

## Подготовка к расчёту прибыли

### Сохраним все ключевые значения для расчётов в отдельных переменных

В условии указано, что бюджет на разработку скважин в регионе составляетс 10млрд рублей или 10 000 000тыс

In [11]:
BUDGET = 10**7

Доход с каждой единицы продукта (тысяча баррелей) составляет 450 000 рублей

In [12]:
INCOME_FROM_ONE = 450

Вероятность убытков должна быть меньше 2,5%

In [13]:
probability_of_losses = 0.025

С помощью машинного обучения необходимо выбрать 200 лучших точек для разработки из 500 для каждого региона. Значит, бюджет на разработку одной скважины

In [14]:
BUDGET_FOR_ONE = BUDGET/200

A значит, минимальный объём скважины должен быть не менее

In [15]:
min_vol_for_one = BUDGET_FOR_ONE/INCOME_FROM_ONE

Ещё нам необходимы будут размеры средних запасов для каждого региона

In [16]:
OIL_RESERVES_1 = well_1['product'].mean()
OIL_RESERVES_2 = well_2['product'].mean()
OIL_RESERVES_3 = well_3['product'].mean()

### Посмотрим, что получилось

In [19]:
print("Бюджет на разработку в регионе", BUDGET/1000, "млн.руб" )
print("Доход с тысячи баррелей", INCOME_FROM_ONE, 'тыс.руб')
print("Критическая вероятность уровня убытков", probability_of_losses*100, "%" )
print("Бюджет на одну скважину", BUDGET_FOR_ONE/1000000, "млн.руб")
print('Минимальный объём неубыточной скважины ', np.ceil(min_vol_for_one), 'тыс. баррелей')
print("Средние запасы для каждого региона (млн. баррелей):", int(OIL_RESERVES_1), int(OIL_RESERVES_2), int(OIL_RESERVES_3), sep = '\n')

Бюджет на разработку в регионе 10000.0 млн.руб
Доход с тысячи баррелей 450 тыс.руб
Критическая вероятность уровня убытков 2.5 %
Бюджет на одну скважину 0.05 млн.руб
Минимальный объём неубыточной скважины  112.0 тыс. баррелей
Средние запасы для каждого региона (млн. баррелей):
92
68
95


### Напишем функцию расчёта прибыли

In [31]:
def profit(target, predictions):
    data = {'predictions':predictions.values, 'target':target.values}
    top = pd.DataFrame(data)
    top = top.sort_values(by ='predictions',ascending=False)
    top200_predictions = top['predictions'].iloc[:200]
    top200_target =top['target'].iloc[:200]
    revenue = INCOME_FROM_ONE*top200_target.sum()
    return revenue - BUDGET

Как видим, средние запасы для каждого региона по результатам 500 скважин ниже, чем минимальный объём неубыточной скважины. Посмотрим, что получится, когда мы отберём 200 лучших.

## Расчёт прибыли и рисков 

### Применим технику Bootstrap c 1000 выборок, чтобы найти распределение прибыли

In [32]:
state = np.random.RandomState(1234)

In [33]:
def bootstrap_revenue(predictions, target):
    revenue = []
    predictions_s =pd.Series(predictions, index=target.index)
    for i in range(1000):  
        target_subsample = target.sample(n = 500, replace = True, random_state=state)
        predictions_subsample =predictions_s[target_subsample.index]
        revenue.append(profit(target_subsample, predictions_subsample))
    revenue = pd.Series(revenue)
    lower = revenue.quantile(q=0.025)
    high = revenue.quantile(q=0.975)
    mean = revenue.mean()
    confidence_interval = st.t.interval(0.95, len(revenue)-1, revenue.mean(), revenue.sem())
    loss = st.percentileofscore(revenue, 0)
    print( '***')
    print('Прибыль в регионе будет составлять (вероятность = 95%) от: {:.1f} млн.руб до: {:.1f} млн.руб'.format((lower/1000),(high/1000)))
    print('Среднее значение прибыли в регионе {:.1f} млн.руб'.format(mean/1000))
    print('95-процентный доверительный интервал находится в пределах', confidence_interval, 'тыс. руб')
    print ('Вероятность убытка в процентах {:.3}'.format(loss))

In [36]:
print("регион 1")
bootstrap_revenue(w1_predict, w1_target_valid)

регион 1
***
Прибыль в регионе будет составлять (вероятность = 95%) от: -78.6 млн.руб до: 902.7 млн.руб
Среднее значение прибыли в регионе 413.3 млн.руб
95-процентный доверительный интервал находится в пределах (397793.7230647271, 428738.58583341615) тыс. руб
Вероятность убытка в процентах 4.4


In [34]:
print("регион 2")
bootstrap_revenue(w2_predict, w2_target_valid)

регион 2
***
Прибыль в регионе будет составлять (вероятность = 95%) от: 32.5 млн.руб до: 847.4 млн.руб
Среднее значение прибыли в регионе 429.1 млн.руб
95-процентный доверительный интервал находится в пределах (416354.5517269733, 441815.7440858635) тыс. руб
Вероятность убытка в процентах 1.9


In [35]:
print("регион 3")
bootstrap_revenue(w3_predict, w3_target_valid)

регион 3
***
Прибыль в регионе будет составлять (вероятность = 95%) от: -142.8 млн.руб до: 855.5 млн.руб
Среднее значение прибыли в регионе 350.1 млн.руб
95-процентный доверительный интервал находится в пределах (334206.37146636925, 365924.7709845834) тыс. руб
Вероятность убытка в процентах 7.9


Как видим, самым удачным по показателям является регион №2, так как доверительный интервал и среднее значение прибыли у него самые высокие, а вероятность убытка самая низкая. 

Вывод:   
После загрузки данных по трём регионам, была проведена проверка на наличие пропусков и дубликатов. Данные были разбиты на обучающую и валидационную выборки.Была построена модель линейной регрессии, которая должна предсказывать запасы нефти. Для каждого региона были найдены метрики RMSE. Самым низким это значение оказалось у второго региона и составила 0,894. Было посчитано минимальное среднее количество продукта в месторождениях региона, достаточное для разработки. Оно составило 112 тыс баррелей. Была проведена процедура бутстрепа, по результатам которого лучшие показатели прибыли, доверительного интервала и вероятность убытка оказались у региона №2. Поэтому регион, который мы предлагаем  для разработки месторождения - второй.
